# Using the MDai API
### Get annotations, create a Pandas Dataframe, create csv, import new labels.

In [1]:
!pip3 install -q --upgrade mdai

You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
import mdai
mdai.__version__

### We need some variables.  
- DOMAIN is the base portion of the project url eg. company.md.ai
- ACCESS_TOKEN can be obtained from User Icon -> User Settings -> Personal Access Tokens
- PROJECT_ID is shown via the info icon on the left of the Annotator

In [ ]:
DOMAIN = 'Fill this in with the appropriate value'
ACCESS_TOKEN = 'Fill this in with the appropriate value'
PROJECT_ID = 'Fill this in with the appropriate value'

### Create the MDai Client

In [ ]:
mdai_client = mdai.Client(domain=DOMAIN, access_token=ACCESS_TOKEN)

### Download the annotations for the project

In [ ]:
mdai_client.project(PROJECT_ID, annotations_only=True)

In [2]:
import json
import pandas as pd

###  Another variable - JSON_FILE
- The project you create prints out the filename of the json annotations file.
- Insert that into JSON_FILE variable

In [3]:
JSON_FILE = 'Fill this in with the appropriate value'
with open(JSON_FILE, 'r') as f:
    data = json.load(f)

In [6]:
def unpackDictionary(df, column):
    ret = None
    ret = pd.concat([df, pd.DataFrame((d for idx, d in df[column].iteritems()))], axis=1)
    del ret[column]
    return ret

a = pd.DataFrame([])
studies = pd.DataFrame([])

# Gets annotations for all datasets
for d in data['datasets']:
    annotations = pd.DataFrame(d['annotations'])
    annotations['dataset'] = d['name']
    study = pd.DataFrame(d['studies'])
    a = a.append(annotations,ignore_index=True)
    studies = studies.append(study,ignore_index=True)

studies = studies[['StudyInstanceUID', 'number']]
g = pd.DataFrame(data['labelGroups'])

#unpack arrays
result = pd.DataFrame([(d, tup.id, tup.name) for tup in g.itertuples() for d in tup.labels])
result.columns = ['labels','id','name']

labelGroups = unpackDictionary(result, 'labels')
labelGroups = labelGroups[['id','name','annotationMode']]
labelGroups.columns = ['groupId','labelId','groupName','labelName','annotationMode']

a = a.merge(labelGroups, on='labelId')
a = a.merge(studies, on='StudyInstanceUID')
a.head()

,SOPInstanceUID,SeriesInstanceUID,StudyInstanceUID,annotationNumber,createdAt,createdById,data,height,id,labelId,...,reviewsPositiveCount,updatedAt,updatedById,width,dataset,groupId,groupName,labelName,annotationMode,number
0,NaN,NaN,2.16.840.1.114274.4504117462496947907456710035...,None,2018-12-03T16:43:14.886Z,U_KymW3E,None,None,A_a4dnoV,L_2l2P0B,...,0,2018-12-03T16:43:14.894Z,U_KymW3E,None,Dataset,G_76xRla,Group 2,Neck,None,29
1,NaN,NaN,2.16.840.1.114274.4504117462496947907456710035...,None,2018-12-03T16:43:11.872Z,U_KymW3E,None,None,A_W497pV,L_8logl7,...,0,2018-12-03T16:43:11.879Z,U_KymW3E,None,Dataset,G_76xRla,Group 2,Head,None,29
2,NaN,NaN,2.16.840.1.114274.4504117462496947907456710035...,None,2018-12-04T17:24:46.380Z,U_6y5LdV,None,None,A_YNjoA4,L_rj6Vjm,...,0,2018-12-04T17:24:46.468Z,U_6y5LdV,None,Dataset,G_gnD1lm,Group 3,Head,None,29
3,NaN,NaN,2.16.840.1.114274.6360863111461398062925621323...,None,2018-12-03T16:25:03.432Z,U_KymW3E,None,None,A_OrPZNk,L_2l2P0B,...,0,2018-12-03T16:25:03.440Z,U_KymW3E,None,Dataset,G_76xRla,Group 2,Neck,None,20
4,NaN,NaN,2.16.840.1.114274.6360863111461398062925621323...,None,2018-12-04T17:23:55.043Z,U_6y5LdV,None,None,A_KVpjz4,L_Mj51jN,...,0,2018-12-04T17:23:55.049Z,U_6y5LdV,None,Dataset,G_gnD1lm,Group 3,Neck,None,20


In [7]:
a.columns

Index(['SOPInstanceUID', 'SeriesInstanceUID', 'StudyInstanceUID',
       'annotationNumber', 'createdAt', 'createdById', 'data', 'height', 'id',
       'labelId', 'machineLearningModelId', 'note', 'radlexTagIds',
       'reviewsNegativeCount', 'reviewsPositiveCount', 'updatedAt',
       'updatedById', 'width', 'dataset', 'groupId', 'groupName', 'labelName',
       'annotationMode', 'number'],
      dtype='object')

# Create csv

In [ ]:
a.to_csv("annotations.csv", index=False)

# Importing Annotations or Predictions

### Importing labels, we need some more variables
- LABEL_ID - create a new label, show id using Label Controls, copy and fill in
- DATASET_ID s shown via the info icon on the left of the Annotator
- MODEL_ID - go to the Models tab on the left, create a new Model and name it or use the id for a prior model

In [ ]:
LABEL_ID = 'Fill this in with the appropriate value'
DATASET_ID = 'Fill this in with the appropriate value'
MODEL_ID = 'Fill this in with the appropriate value'

### Create subset

In [ ]:
#For example, get all exams with MLA
subset = a[~a.machineLearningModelId.isnull()]

### Create imported annotations dictionary
- Use correct format for type and scope of annotation|

In [ ]:
#For example, this is a global label at the exam level
annotations = []
for i,row in subset.iterrows():
  annotations.append( {
    'labelId': LABEL_ID,
    'StudyInstanceUID': row.StudyInstanceUID
  })
  
len(annotations)

### Or use this for bounding boxes

In [ ]:
# local image-scoped label where annotation mode is 'bbox' (Bounding Box)
annotations = []
for i,row in subset.iterrows():
  annotations.append( {
    'labelId': LABEL_ID,
    'SOPInstanceUID': row.StudyInstanceUID,
    'data': {'x': 200, 'y': 200, 'width': 200, 'height': 400}
  })
  
len(annotations)

### Import your annotations
- If all works, you should see the labels in your project Progress 

In [ ]:
#Import
mdai_client.load_model_annotations(PROJECT_ID, DATASET_ID, MODEL_ID, annotations)